In [ ]:
import itk
import vtk
import os

In [75]:
dicoms_dir = 'data/pulmonary'
image_3d_path = 'data/3d_pulmonary.mha' 
image_3d_seg_path = 'data/3d_pulmonary_seg.mha' 

# 1. Read image

In [66]:
def DICOMsTo3Dimage(data_dir, out_file):
    namesGenerator = itk.GDCMSeriesFileNames.New()
    namesGenerator.SetUseSeriesDetails(True)
    namesGenerator.AddSeriesRestriction("0008|0021")
    namesGenerator.SetGlobalWarningDisplay(False)
    namesGenerator.SetDirectory(data_dir)

    seriesUID = namesGenerator.GetSeriesUIDs()

    assert len(seriesUID), "No DICOMs in: " + data_dir

    seriesIdentifier = seriesUID[0]
    fileNames = namesGenerator.GetFileNames(seriesIdentifier)

    reader = itk.ImageSeriesReader.New(
        ImageIO=itk.GDCMImageIO.New(),
        FileNames=fileNames,
        ForceOrthogonalDirection=False
    )

    writer = itk.ImageFileWriter.New(
        Input=reader,
        FileName=out_file,
        UseCompression=True
    )
    writer.Update()

In [70]:
# Read DICOM series
if not os.path.isfile(image_3d_path):
    DICOMsTo3Dimage(dicoms_dir, image_3d_path)
    print("Created 3d image from dcm series")

# Read 3d image
image = itk.imread(image_3d_path)

print("Image size: ", itk.size(image))
print("Image type: ", itk.template(image)[-1])

Image size:  itkSize3 ([888, 1100, 28])
Image type:  (<itkCType signed short>, 3)


# 2. Preprocessing

In [ ]:
# Clipping
cropper = itk.ExtractImageFilter.New(Input=image)
cropper.SetDirectionCollapseToIdentity()
extraction_region = cropper.GetExtractionRegion()
extraction_region.SetIndex([235, 420, 1])
extraction_region.SetSize([410, 290, 25])
cropper.SetExtractionRegion(extraction_region)
cropper.Update()

# Normalize image
img_normalized = itk.RescaleIntensityImageFilter.New(
    Input=cropper,
    OutputMinimum=0,
    OutputMaximum=255
)

# Bluring
img_normalized_f = itk.CastImageFilter[itk.Image[itk.SS,3], itk.Image[itk.F,3]].New(Input=img_normalized)
diffusion = itk.GradientAnisotropicDiffusionImageFilter.New(Input=img_normalized_f, TimeStep=0.005)

# 3. KMeans segmentation

In [72]:
def KMeans_segmetation(image, label, apply_filter=False, region=None):
    # Apply KMean segmentation
    kmeans = itk.ScalarImageKmeansImageFilter.New(Input=image, ImageRegion=region)
    kmeans.AddClassWithInitialMean(0)
    kmeans.AddClassWithInitialMean(70)
    kmeans.AddClassWithInitialMean(150)
    kmeans.AddClassWithInitialMean(250)
    kmeans.Update()

    # Create label map
    kmeans_uc = itk.CastImageFilter[itk.Image[itk.SS,3], itk.Image[itk.UC,3]].New(Input=kmeans)
    kmeans_uc.Update()
    label_map = itk.LabelImageToLabelMapFilter.New(Input=kmeans_uc)
    label_map.Update()

    # Extract labled pixels
    obj = itk.LabelMapMaskImageFilter.New(Input=label_map, FeatureImage=kmeans, Label=label)
    obj.Update()

    # Apply denoise filter
    result = obj
    if apply_filter:
        filtered_obj = itk.MedianImageFilter.New(Input=obj, Radius=[1, 1, 1])
        filtered_obj.Update()
        result = filtered_obj
    
    return result

In [73]:
# Extract sceleton by KMeans segmentation
skeleton_region = diffusion.GetOutput().GetLargestPossibleRegion()
skeleton_region.SetIndex([235, 420, 1])
skeleton_region.SetSize([410, 289, 25])
skeleton = KMeans_segmetation(image=diffusion, label=3, apply_filter=0, region=skeleton_region)

# Extract lungs by KMeans segmentation
lungs_region    = diffusion.GetOutput().GetLargestPossibleRegion()
lungs_region.SetIndex([250, 450,  1])
lungs_region.SetSize([ 380, 200, 25])
lungs    = KMeans_segmetation(image=diffusion, label=1, apply_filter=1, region=lungs_region)

# Merge sceleton and lungs masks
skeleton_lungs = itk.AddImageFilter.New(Input1=lungs, Input2=skeleton)

# 4. Add unsegmented values as background 

In [76]:
# Set background mask
background_mask = itk.BinaryThresholdImageFilter.New(
    Input=skeleton_lungs,
    LowerThreshold=1,
    InsideValue=0,
    OutsideValue=1,
)

# Set the background values to be different from the segment values
background = itk.RescaleIntensityImageFilter.New(
    Input=img_normalized,
    OutputMinimum=4,
    OutputMaximum=255
)

# Separate the background squeaks from the general  
background_masked = itk.MultiplyImageFilter.New(
    Input1=background,
    Input2=background_mask,
)

# Combine segments and background
join = itk.AddImageFilter.New(
    Input1=skeleton_lungs,
    Input2=background_masked,
)

itk.imwrite(join, image_3d_seg_path)

# 5. VTK visualization

In [78]:
def Slider(interactor, range, x, y, title, default_value=None, callback=lambda x: x):
    # Set slider properties
    slider = vtk.vtkSliderRepresentation2D()
    slider.SetTitleText(title)
    slider.SetMinimumValue(range[0])
    slider.SetMaximumValue(range[-1])
    slider.SetValue(default_value)
    slider.ShowSliderLabelOn()
    slider.SetSliderWidth(0.03)
    slider.SetSliderLength(0.0001)
    slider.SetEndCapWidth(0)
    slider.SetTitleHeight(0.02)
    slider.SetTubeWidth(0.005)
    
    # Set the slider position
    slider.GetPoint1Coordinate().SetCoordinateSystemToNormalizedDisplay()
    slider.GetPoint1Coordinate().SetValue(x, y)
    slider.GetPoint2Coordinate().SetCoordinateSystemToNormalizedDisplay()
    slider.GetPoint2Coordinate().SetValue(x + 0.25, y)

    # Add the slider to the UI
    sliderWidget = vtk.vtkSliderWidget()
    sliderWidget.SetInteractor(interactor)
    sliderWidget.SetRepresentation(slider)
    sliderWidget.EnabledOn()
    
    # Add callback
    def _cb(s, *args):
        slider_representation = s.GetSliderRepresentation()
        value = slider_representation.GetValue()
        callback(value)
    sliderWidget.AddObserver("InteractionEvent", _cb)
    return sliderWidget

In [79]:
image = itk.vtk_image_from_image(itk.imread(image_3d_seg_path))

mapper = vtk.vtkGPUVolumeRayCastMapper()
mapper.SetInputData(image)

volume = vtk.vtkVolume()
volume.SetMapper(mapper)
volumeProperty = volume.GetProperty()

GREEN = (0, 1, 0)
BLUE  = (0, 0, 1)
BLACK = (0, 0, 0)
WHITE = (1, 1, 1)

colorDefault = vtk.vtkColorTransferFunction()
colorDefault.AddRGBPoint(0, *BLACK)
colorDefault.AddRGBPoint(1, *GREEN)
colorDefault.AddRGBPoint(2, *BLACK)
colorDefault.AddRGBPoint(3, *BLUE)
colorDefault.AddRGBSegment(4, *BLACK, 255, *WHITE)

opacityDefault = vtk.vtkPiecewiseFunction()
opacityDefault.AddPoint(0, 0)
opacityDefault.AddPoint(1, 0.05)
opacityDefault.AddPoint(2, 0)
opacityDefault.AddPoint(3, 0)
opacityDefault.AddSegment(4, 0., 255, 0.01)

volumeProperty.SetColor(colorDefault)
volumeProperty.SetScalarOpacity(opacityDefault)
volumeProperty.SetInterpolationTypeToNearest()

renderer = vtk.vtkRenderer()
renderer.AddVolume(volume)

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renderWindow)

cb_opacity_lungs = lambda x : opacityDefault.AddPoint(1, x)
cb_opactiy_sceleton = lambda x : opacityDefault.AddPoint(3, x)
cb_opacity_background = lambda x : opacityDefault.AddSegment(4, 0., 255, x)

s1 = Slider(interactor=interactor, range=(0, 1), x=0.7, y=0.45, title="Lungs opacity",       default_value=0.05, callback=cb_opacity_lungs)
s2 = Slider(interactor=interactor, range=(0, 1), x=0.7, y=0.30, title="Sceleton opacity",    default_value=0,    callback=cb_opactiy_sceleton)
s3 = Slider(interactor=interactor, range=(0, 1), x=0.7, y=0.15, title="Background opacity",  default_value=0.01, callback=cb_opacity_background)

renderWindow.SetSize(500, 200)
renderWindow.Render()
interactor.Start()